In [ ]:
# parameters_dynamic2.ipynb

import geopandas as gdp
import numpy as np
import pandas as pd
import itertools
import sys, trace
import matplotlib.pyplot as plt
from scenarios import scenarios  # Import scenario configurations
from scipy.spatial.distance import cdist
import os


In [ ]:

# Select Scenario
scenario_name = "baseline2"  # Change this to switch scenarios
params = scenarios[scenario_name]  # Load selected scenario parameters


In [ ]:

# Load the GeoJSON file
# location_nodes = gdp.read_file("location_nodes.geojson")
#location_nodes = gdp.read_file("location_refcamps.geojson")
location_nodes =  gdp.read_file(params["location_file"])
location_nodes

,Name,Camp,type_f,id,geometry
0,Nguenyyiel 1,Nguenyyiel,demand_point,dp1,POINT (34.3348 8.29523)
1,Nguenyyiel 2,Nguenyyiel,demand_point,dp2,POINT (34.31575 8.3153)
2,Nguenyyiel 3,Nguenyyiel,demand_point,dp3,POINT (34.32889 8.30275)
3,Nguenyyiel 4,Nguenyyiel,demand_point,dp4,POINT (34.31347 8.32582)
4,Nguenyyiel 5,Nguenyyiel,demand_point,dp5,POINT (34.32482 8.31467)
5,Kule 1,Kule,demand_point,dp6,POINT (34.25027 8.27879)
6,Kule 2,Kule,demand_point,dp7,POINT (34.24838 8.30246)
7,Kule 3,Kule,demand_point,dp8,POINT (34.25992 8.29336)
8,Tierkidi 1,Tierkidi,demand_point,dp9,POINT (34.27671 8.26016)
9,Tierkidi 2,Tierkidi,demand_point,dp10,POINT (34.28039 8.27549)


In [4]:

# Add x and y coordinates
location_nodes.loc[:, 'x'] = location_nodes.geometry.x
location_nodes.loc[:, 'y'] = location_nodes.geometry.y

# Sort 
location_nodes = location_nodes.sort_values(by=['y', 'x']).reset_index(drop=True)

# Label sorted demand points
demand_points_gdf = location_nodes.loc[location_nodes.type_f == "demand_point"].copy()
demand_points_gdf['label'] = ['i' + str(i + 1) for i in range(len(demand_points_gdf))]

# Save demand point labels to a Numpy Array
dps = demand_points_gdf['label'].to_numpy()

# Subset location types
hps_gdf = location_nodes[location_nodes.type_f == "HP"]
hcs_gdf = location_nodes[location_nodes.type_f == "HC"]
hfs_gdf = location_nodes[(location_nodes.type_f == "HC") | (location_nodes.type_f == "HP")].drop_duplicates(subset='geometry').reset_index(drop=False)

# Label candidate locations
hfs_gdf['label'] = ['j' + str(j + 1) for j in range(len(hfs_gdf))]

# Save location labels
hfs = hfs_gdf['label'].to_numpy()
hps = hfs_gdf[hfs_gdf['geometry'].isin(hps_gdf['geometry'])]['label'].to_numpy()
hcs = hfs_gdf[hfs_gdf['geometry'].isin(hcs_gdf['geometry'])]['label'].to_numpy()
 

In [5]:
demand_points_gdf

,Name,Camp,type_f,id,geometry,x,y,label
1,Pinyudo-II 1,Pinyudo-II,demand_point,dp16,POINT (34.4282 7.75013),34.428196,7.750127,i1
3,Pinyudo 1,Pinyudo,demand_point,dp17,POINT (34.15395 7.91722),34.153949,7.917223,i2
5,Pinyudo 2,Pinyudo,demand_point,dp18,POINT (34.14292 7.92182),34.142925,7.921820,i3
6,Pinyudo 3,Pinyudo,demand_point,dp19,POINT (34.15634 7.92616),34.156337,7.926163,i4
8,Jewi 1,Jewi,demand_point,dp13,POINT (34.72145 8.13262),34.721450,8.132619,i5
9,Jewi 2,Jewi,demand_point,dp14,POINT (34.73359 8.13417),34.733586,8.134171,i6
12,Jewi 3,Jewi,demand_point,dp15,POINT (34.72405 8.14326),34.724051,8.143258,i7
13,Tierkidi 1,Tierkidi,demand_point,dp9,POINT (34.27671 8.26016),34.276707,8.260158,i8
17,Tierkidi 3,Tierkidi,demand_point,dp11,POINT (34.27299 8.27071),34.272994,8.270714,i9
18,Tierkidi 2,Tierkidi,demand_point,dp10,POINT (34.28039 8.27549),34.280388,8.275491,i10


In [ ]:
camps = set(location_nodes["Camp"].unique())
camps
camp_demand_labels = demand_points_gdf.groupby("Camp")["label"].apply(set).to_dict()
camp_demand_labels
camp_candidate_location_labels = hfs_gdf.groupby("Camp")["label"].apply(set).to_dict()
camp_candidate_location_labels 

{'Jewi': {'j5'},
 'Kule': {'j9'},
 'Nguenyyiel': {'j10', 'j11', 'j8'},
 'Pinyudo': {'j3', 'j4'},
 'Pinyudo-II': {'j1', 'j2'},
 'Tierkidi': {'j6', 'j7'}}

In [ ]:
# This is _just_ to have insights on which values would be adequate for t'max and t''max 

from shapely.geometry import Point

# Initialize dictionaries to store results
avg_dist_demand_to_hp = {}
avg_dist_demand_to_hc = {}
min_intercamp_distance = {} # For distances between different camps
max_withincamp_distance = {} # For maximum distance within the same camp

# Compute distances for each camp
for camp in camps:
    # Filter locations by camp
    camp_demand_points = demand_points_gdf[demand_points_gdf["Camp"] == camp]
    camp_hps = hps_gdf[hps_gdf["Camp"] == camp]
    camp_hcs = hcs_gdf[hcs_gdf["Camp"] == camp]

    # Compute distances between demand points and HPs
    if not camp_hps.empty:
        avg_distances_per_demand_point_to_hp = camp_demand_points.to_crs(epsg=3857).geometry.apply(lambda dp: camp_hps.to_crs(epsg=3857).geometry.distance(dp).mean())
        avg_dist_demand_to_hp[camp] = avg_distances_per_demand_point_to_hp.mean()
    else:
        avg_dist_demand_to_hp[camp] = None  # No HPs in this camp

    # Compute distances between demand points and HCs
    if not camp_hcs.empty:
        avg_distances_per_demand_point_to_hc = camp_demand_points.to_crs(epsg=3857).geometry.apply(lambda dp: camp_hcs.to_crs(epsg=3857).geometry.distance(dp).mean())
        avg_dist_demand_to_hc[camp] = avg_distances_per_demand_point_to_hc.mean()
    else:
        avg_dist_demand_to_hc[camp] = None  # No HCs in this camp

    # Now calculate maximum within-camp distance (distance between any two locations within the same camp)
    locations_camp = location_nodes[location_nodes["Camp"] == camp]
    
    # Reproject to EPSG:3857 (meters) and compute the maximum pairwise distance within the camp
    max_within_distance = locations_camp.to_crs(epsg=3857).geometry.apply(
        lambda loc: locations_camp.to_crs(epsg=3857).geometry.distance(loc).max()
    ).max()
    
    # Store the maximum distance within the camp
    max_withincamp_distance[camp] = max_within_distance


# Compute minimum distance between any location in different camps
for camp1 in camps:
    for camp2 in camps:
        if camp1 != camp2:
            locations_camp1 = location_nodes[location_nodes["Camp"] == camp1]
            locations_camp2 = location_nodes[location_nodes["Camp"] == camp2]
            min_distance = locations_camp1.to_crs(epsg=3857).geometry.apply(lambda loc: locations_camp2.to_crs(epsg=3857).geometry.distance(loc).min()).min()
            min_intercamp_distance[(camp1, camp2)] = min_distance

# Now find the overall minimum distance across all inter-camp pairs
overall_min_distance = min(min_intercamp_distance.values())

# Find the overall maximum within-camp distance
overall_max_withincamp_distance = max(max_withincamp_distance.values())


# # Convert results to DataFrames for better visualization
# df_avg_dist_hp = pd.DataFrame(list(avg_dist_demand_to_hp.items()), columns=["Camp", "Avg_Dist_Demand_to_HP"])
# df_avg_dist_hc = pd.DataFrame(list(avg_dist_demand_to_hc.items()), columns=["Camp", "Avg_Dist_Demand_to_HC"])
# df_min_intercamp = pd.DataFrame(list(overall_min_distance.items()), columns=["Camp_Pair", "Min_Distance"])
# df_max_withincamp = pd.DataFrame(list(overall_max_withincamp_distance.items()), columns=["Camp", "Max_Distance"])

# Display results
print(avg_dist_demand_to_hp)
print(avg_dist_demand_to_hc)
print(min_intercamp_distance)
print(overall_min_distance)
print(max_withincamp_distance)
print(overall_max_withincamp_distance)



{'Pinyudo': np.float64(1216.529500210322), 'Jewi': np.float64(899.632630625119), 'Tierkidi': np.float64(1328.089141904581), 'Nguenyyiel': np.float64(1736.9335833532446), 'Pinyudo-II': np.float64(210.93550443474055), 'Kule': np.float64(1297.4795165741423)}
{'Pinyudo': np.float64(1122.0924704762308), 'Jewi': np.float64(899.632630625119), 'Tierkidi': np.float64(1209.715886320969), 'Nguenyyiel': np.float64(2349.233464022447), 'Pinyudo-II': np.float64(171.662282832902), 'Kule': np.float64(1297.4795165741423)}
{('Pinyudo', 'Jewi'): np.float64(66662.49778761216), ('Pinyudo', 'Tierkidi'): np.float64(39631.59356051772), ('Pinyudo', 'Nguenyyiel'): np.float64(45737.26937417198), ('Pinyudo', 'Pinyudo-II'): np.float64(35773.985453731264), ('Pinyudo', 'Kule'): np.float64(40791.55703534717), ('Jewi', 'Pinyudo'): np.float64(66662.49778761216), ('Jewi', 'Tierkidi'): np.float64(50308.168394830536), ('Jewi', 'Nguenyyiel'): np.float64(46580.85522272594), ('Jewi', 'Pinyudo-II'): np.float64(53788.6184507043

In [30]:

def compute_distance_matrix(demand_points_gdf, hfs_gdf):
    """
    Compute the distance matrix between demand points and candidate health facility locations.

    Parameters:
    - demand_points_gdf: GeoDataFrame containing demand points with 'geometry'.
    - hfs_gdf: GeoDataFrame containing candidate health facility locations with 'geometry'.

    Returns:
    - distance_matrix: 2D NumPy array of distances (rows: demand points, columns: health facilities).
    """
    # Extract coordinates as NumPy arrays directly from the geometry column
    demand_coords = np.array(demand_points_gdf.geometry.apply(lambda point: (point.x, point.y)).tolist())
    hfs_coords = np.array(hfs_gdf.geometry.apply(lambda point: (point.x, point.y)).tolist())
    
    # Compute the distance matrix using cdist with Euclidean metric
    distance_matrix = cdist(demand_coords, hfs_coords, metric='euclidean')
    
    # Create a labeled DataFrame
    distance_df = pd.DataFrame(distance_matrix, index=dps, columns=hfs)

    return distance_df

In [32]:

# Example usage:
distance_df = compute_distance_matrix(demand_points_gdf, hfs_gdf)

# To save the above matrix into an Excel file to subsequently read
# distance_df.to_excel('distance_matrix_refcamps.xlsx', sheet_name='DistanceMatrixRefCamps')#, float_format="%.2f")

# Distance matrix
# distance_matrix = pd.read_excel('distance_matrix_ij.xlsx', index_col=0)
# distance_matrix = pd.read_excel('distance_matrix_refcamps.xlsx', index_col=0)
distance_df

,j1,j2,j3,j4,j5,j6,j7,j8,j9,j10,j11
i1,0.001531,0.001878,0.331254,0.321705,0.492276,0.540610,0.541792,0.554215,0.576201,0.575808,0.588757
i2,0.321305,0.320558,0.011156,0.011567,0.614027,0.368667,0.371249,0.419419,0.393440,0.430803,0.438772
i3,0.333107,0.332378,0.003442,0.017605,0.622647,0.367971,0.370603,0.420157,0.391888,0.430930,0.438579
i4,0.324080,0.323243,0.015570,0.003508,0.608567,0.359430,0.362015,0.410322,0.384187,0.421631,0.429570
i5,0.483470,0.480263,0.616987,0.598138,0.010836,0.470945,0.469268,0.420926,0.498663,0.442501,0.454440
i6,0.492157,0.488970,0.628913,0.610076,0.012194,0.482158,0.480455,0.431548,0.509622,0.452958,0.464792
i7,0.493518,0.490302,0.623187,0.604302,0.001051,0.470526,0.468792,0.419327,0.497697,0.440607,0.452367
i8,0.533193,0.530421,0.367056,0.352855,0.463407,0.009578,0.009859,0.067368,0.046034,0.069227,0.075025
i9,0.544368,0.541598,0.375602,0.361650,0.469762,0.004612,0.001749,0.065765,0.035300,0.063732,0.067781
i10,0.546915,0.544115,0.382682,0.368544,0.463995,0.013257,0.010220,0.057118,0.037164,0.055138,0.059800


In [33]:
import numpy as np

def compute_distance_matrix_meters(demand_points_gdf, hfs_gdf, crs_epsg=3857):
    """
    Compute the distance matrix between demand points and candidate health facility locations.

    Parameters:
    - demand_points_gdf: GeoDataFrame containing the demand points with geometry (usually point geometries).
    - hfs_gdf: GeoDataFrame containing the candidate health facility locations with geometry.
    - crs_epsg: The EPSG code to which the geometries will be reprojected. Default is 3857 (Web Mercator).

    Returns:
    - distance_df: A pandas DataFrame where the rows are demand points, the columns are health facilities,
                   and the values are the distances between them.
    """
    # Reproject both demand points and health facilities to the target CRS (e.g., EPSG:3857 for meters)
    demand_points_gdf = demand_points_gdf.to_crs(epsg=crs_epsg)
    hfs_gdf = hfs_gdf.to_crs(epsg=crs_epsg)

    # Initialize an empty distance matrix with dimensions (num_demand_points x num_health_facilities)
    num_demand_points = len(demand_points_gdf)
    num_health_facilities = len(hfs_gdf)
    distance_matrix = np.zeros((num_demand_points, num_health_facilities))

    # Compute distances
    for i, demand_point in enumerate(demand_points_gdf.geometry):
        for j, hf_location in enumerate(hfs_gdf.geometry):
            distance_matrix[i, j] = demand_point.distance(hf_location)
    
    # Create a DataFrame with labeled indices and columns
    distance_df = pd.DataFrame(distance_matrix, index=dps, columns=hfs) 


    return distance_df


In [38]:

# Example usage:
distance_df = compute_distance_matrix_meters(demand_points_gdf, hfs_gdf)
distance_df

# To save the above matrix into an Excel file to subsequently read
# distance_df.to_excel('distance_matrix_refcamps_meters.xlsx', sheet_name='DistanceMatrixRefCamps')#, float_format="%.2f")

# Distance matrix
# distance_matrix = pd.read_excel('distance_matrix_ij.xlsx', index_col=0)
distance_matrix = pd.read_excel('distance_matrix_refcamps_meters.xlsx', index_col=0)
distance_matrix

,j1,j2,j3,j4,j5,j6,j7,j8,j9,j10,j11
i1,171.662283,210.935504,36964.973517,35914.797190,55138.744518,60722.905988,60857.458005,62287.362917,64717.274713,64711.071933,66164.980985
i2,35860.433038,35773.985454,1242.010560,1297.054743,68445.002007,41411.811038,41700.892036,47074.761109,44212.904723,48371.654060,49275.979456
i3,37176.018594,37091.590227,386.867395,1961.648856,69399.626668,41325.517067,41620.150145,47147.208946,44032.041204,48376.365135,49245.338880
i4,36178.764521,36082.144396,1737.399457,390.884901,67830.867328,40374.307901,40663.667441,46052.607106,43173.618482,47341.209864,48242.287331
i5,54148.748503,53788.618451,68764.991645,66662.497788,1217.198687,52469.710665,52284.562486,46930.955667,55574.889463,49346.650243,50684.837927
i6,55112.656438,54754.747586,70092.199856,67990.999212,1364.478502,53715.901569,53527.791366,48110.295410,56792.280310,50507.276934,51833.587945
i7,55279.177066,54917.963295,69462.702241,67356.171801,117.220703,52416.479891,52224.697948,46744.032516,55459.602284,49126.360930,50444.240857
i8,59892.360976,59580.043535,41216.806854,39631.593561,51620.515242,1072.161375,1106.826257,7522.042343,5161.938896,7758.170480,8421.923490
i9,61148.075402,60835.977447,42182.360786,40624.626053,52333.988802,516.738806,196.558766,7332.595833,3955.241419,7131.532149,7600.614358
i10,61439.364278,61123.960816,42973.756082,41395.612728,51695.581037,1481.997101,1142.478477,6367.221417,4153.856321,6172.024491,6709.391594


In [ ]:
######################################

# Health services and workers
services = ['basic','maternal1','maternal2']
health_workers = ['doctor','nurse','midwife']
levels = ['hp', 'hc']



# Assign scenario parameters
HFs_to_locate = params["HFs_to_locate"]
t1max = params["t1max"]
t2max = params["t2max"]
workers_to_allocate = params["workers_to_allocate"]
working_hours = params["working_hours"]
service_time = params["service_time"]

# Lower bound workers per HF type
lb_workers_df = pd.DataFrame(params["lb_workers"], index=health_workers)
lb_workers = {(health_workers[p], levels[l]): lb_workers_df.iloc[p, l] 
      for p, l in itertools.product(range(len(health_workers)), range(len(levels)))}

# Where can each service be provided?
services_at_HFs_df = pd.DataFrame(params["services_at_HFs"], index=services)
a_HF = {(services[s], levels[l]): services_at_HFs_df.iloc[s, l] 
      for s, l in itertools.product(range(len(services)), range(len(levels)))}

# Which health worker can deliver each service?
services_per_worker_df = pd.DataFrame(params["services_per_worker"], index=health_workers)
a_W = {(health_workers[p], services[s]): services_per_worker_df.iloc[p, s] 
      for p, s in itertools.product(range(len(health_workers)), range(len(services)))}

# Demand rates
total_population = {(key): params["total_population"] for key in dps}

# Opening hours
demand_rate_opening_hours_df = pd.DataFrame([params["demand_rate_opening_hours"]] * len(dps), index=dps, columns=services)
dr_oh = {(dps[i], services[s]): demand_rate_opening_hours_df.iloc[i, s] 
      for i, s in itertools.product(range(len(dps)), range(len(services)))}

dd_oh = {(key): int(round(total_population[i] * dr_oh[key])) for i in dps for key in dr_oh}

# Closing hours
demand_rate_closing_hours_df = pd.DataFrame([params["demand_rate_closing_hours"]] * len(dps), index=dps, columns=services)
dr_ch = {(dps[i], services[s]): demand_rate_closing_hours_df.iloc[i, s] 
      for i, s in itertools.product(range(len(dps)), range(len(services)))}

dd_ch = {(key): int(round(total_population[i] * dr_ch[key])) for i in dps for key in dr_ch}
